In [41]:
from langchain.document_loaders import PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
loader = PDFMinerLoader("/workspaces/documents/Eberron_last_war.pdf")

data = loader.load_and_split(text_splitter=splitter)

In [42]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(data, OpenAIEmbeddings())

In [43]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

retriever = faiss_index.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [44]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [47]:
chain.invoke("What spells can a dusk hag cast?")

'A dusk hag can cast the following spells:\n\n- At will: detect magic, disguise self\n- 3/day each: dream, hypnotic pattern, sleep (9d8)\n- 1/day each: legend lore, scrying'